In [1]:
import mdtraj as mdt
from crossflow.tasks import SubprocessTask
from crossflow.clients import Client
from distributed import LocalCluster

In [82]:
crdfile = '1qy1.ncrst'
topfile = '1qy1.prmtop'

In [2]:
cluster = LocalCluster()

In [3]:
print(cluster)

LocalCluster(c529a204, 'tcp://127.0.0.1:50304', workers=5, threads=10, memory=16.00 GiB)


In [5]:
client = Client(cluster)
print(client.scheduler_info())

{'type': 'Scheduler', 'id': 'Scheduler-ebee1797-1244-48a5-a820-7e0cb3a18d02', 'address': 'tcp://127.0.0.1:50304', 'services': {'dashboard': 8787}, 'started': 1662738143.531083, 'workers': {'tcp://127.0.0.1:50317': {'type': 'Worker', 'id': 4, 'host': '127.0.0.1', 'resources': {}, 'local_directory': '/Users/pazcal/Dropbox/Pazcal/crossflow/sandbox/dask-worker-space/worker-r5miz0f5', 'name': 4, 'nthreads': 2, 'memory_limit': 3435973836, 'last_seen': 1662738195.118521, 'services': {'dashboard': 50318}, 'metrics': {'executing': 0, 'in_memory': 0, 'ready': 0, 'in_flight': 0, 'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}}, 'spilled_nbytes': {'memory': 0, 'disk': 0}, 'cpu': 0.1, 'memory': 99663872, 'time': 1662738195.1022232, 'read_bytes': 6142.904492244673, 'write_bytes': 6142.904492244673, 'read_bytes_disk': 0.0, 'write_bytes_disk': 0.0, 'num_fds': 29}, 'nanny': 'tcp://127.0.0.1:50308'}, 'tcp://127.0.0.1:50320': {'type': 'Worker', 'id': 2, 'host': '127.0.0.1', 'resources': {}, 

In [83]:
md = SubprocessTask('python /users/pazcal/.local/bin/fakemd -c x.ncrst -t x.prmtop -r y.ncrst -x x.nc -o x.log -n 100')
md.set_inputs(['x.ncrst', 'x.prmtop'])
md.set_outputs(['x.log', 'y.ncrst', 'x.nc'])

In [84]:
log, restart, traj = client.submit(md, crdfile, topfile)

In [85]:
t = mdt.load(traj.result(), top=topfile)

In [86]:
print(t)

<mdtraj.Trajectory with 100 frames, 1513 atoms, 387 residues, without unitcells>


In [87]:
log2, restart2, traj2 = client.submit(md, restart, topfile)

In [89]:
t2 = mdt.load(traj2.result(), top=topfile)
print(t2)

<mdtraj.Trajectory with 100 frames, 1513 atoms, 387 residues, without unitcells>


In [90]:
print(mdt.rmsd(t2, t[0]))

[0.4994866  0.50185    0.50540894 0.50739187 0.5097401  0.512653
 0.51434046 0.5171204  0.51926684 0.52151304 0.52392244 0.5265314
 0.52945673 0.53232384 0.5350179  0.5369747  0.5394395  0.541622
 0.5435144  0.5465481  0.547528   0.549905   0.5530044  0.55623204
 0.55806303 0.55957323 0.56215966 0.56555915 0.5681317  0.56882113
 0.57067394 0.57388633 0.576492   0.5787879  0.58157027 0.58362126
 0.5858898  0.5865279  0.58844835 0.5916371  0.592785   0.59464616
 0.59557754 0.5984795  0.59999174 0.60085016 0.6032433  0.605566
 0.60726154 0.60938865 0.61238825 0.61521167 0.6167431  0.6191984
 0.62062144 0.622892   0.624968   0.6274559  0.62949604 0.63206464
 0.63398284 0.6353819  0.63650215 0.6390065  0.6420693  0.6444541
 0.6448696  0.6466705  0.64873725 0.65021133 0.6515565  0.6535436
 0.6551596  0.6568487  0.65857464 0.66053385 0.66195995 0.66398835
 0.66542935 0.66807383 0.66930234 0.67161375 0.6723445  0.6739515
 0.6757766  0.67843306 0.6785253  0.6796452  0.6820652  0.68346006
 0.685

In [92]:
print(t.topology.select('resname PRZ and name N1'))
print(t.topology.select('resid 119 and name OH'))

[1278]
[976]


In [93]:
d = mdt.compute_distances(t2[-1], [[976, 1278]])

In [94]:
print(d)

[[0.6181121]]


In [97]:
def get_distance(crdfile, topfile):
    t = mdt.load(crdfile, top=topfile)
    d = mdt.compute_distances(t[-1], [[976, 1278]])
    return d[0][0]

In [98]:
print(get_distance(crdfile, topfile))

0.254477


In [99]:
logs, restarts, trajs = client.map(md, [restart2] * 5, topfile)

In [101]:
for r in restarts:
    print(get_distance(r.result(), topfile))

1.0961728
0.3932048
1.2195426
0.41828528
1.3939843


In [103]:
distances = [get_distance(r.result(), topfile) for r in restarts]

In [104]:
furthest = distances.index(max(distances))

In [105]:
logs, restarts, trajs = client.map(md, [restarts[furthest]] * 5, topfile)

In [108]:
for r in restarts:
    print(get_distance(r.result(), topfile))

1.7472494
1.8796166
1.2622313
1.4575677
1.6736182


In [109]:
for cycle in range(10):
    logs, restarts, trajs = client.map(md, [restarts[furthest]] * 5, topfile)
    distances = [get_distance(r.result(), topfile) for r in restarts]
    furthest = distances.index(max(distances))
    print(distances[furthest])
    

1.7522186
2.2407331
2.5816925
3.3326123
4.0655303
4.624699
4.951242
5.2860804
5.6091743
5.896824
